In [4]:
# Initial imports
import pandas as pd
import hvplot.pandas
from sklearn.linear_model import LinearRegression
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import skew
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import seaborn as sns

train_path = 'Resources/train.csv'
test_path = 'Resources/test.csv'
trained_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)


In [3]:
from pyspark.sql import SparkSession
# Create a Spark session
spark = SparkSession.builder \
    .appName("YourAppNameHere") \
    .getOrCreate()

# Load the training and test data as Spark DataFrames
train_path = 'Resources/train.csv'
test_path = 'Resources/test.csv'

# You can specify the format and options if needed
train_data = spark.read.csv(train_path, header=True, inferSchema=True)
test_data = spark.read.csv(test_path, header=True, inferSchema=True)

# You can perform various Spark operations on these DataFrames
# For example, to display the first few rows of the training data
train_data.show()



RuntimeError: Java gateway process exited before sending its port number

In [5]:
test_data.show()

AttributeError: 'DataFrame' object has no attribute 'show'

In [6]:
print(trained_data.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [7]:
trained_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [8]:
columns_to_remove = ['FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'MasVnrType', 'Alley']

In [9]:
X = trained_data.copy()
X = X.loc[(X['YrSold'] >= 2006) & (X['YrSold'] <= 2010)] #not necessary, the dataset is already 2006-2010
X.drop(columns=columns_to_remove, inplace=True, axis=1)
Xf = X.dropna()
Xf.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [10]:
dummies = pd.get_dummies(Xf, columns=['MSZoning','LotConfig','LotFrontage','Street','LotShape','LandContour',
                       'Utilities','LandContour','LandSlope','Neighborhood',
                       'Condition1','Condition2','BldgType','HouseStyle','SaleType','RoofStyle',
                      'RoofMatl','Exterior1st','Exterior2nd','ExterQual','ExterCond',
                      'Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
                        'BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual',
                 'Functional','GarageType','GarageFinish','GarageQual',
                  'GarageCond','PavedDrive','SaleType','SaleCondition'], dtype=float)
dummies

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,8450,7,5,2003,2003,196.0,706,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,20,9600,6,8,1976,1976,0.0,978,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,60,11250,7,5,2001,2002,162.0,486,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,70,9550,7,5,1915,1970,0.0,216,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,60,14260,8,5,2000,2000,350.0,655,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,7917,6,5,1999,2000,0.0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1456,1457,20,13175,6,6,1978,1988,119.0,790,163,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1457,1458,70,9042,7,9,1941,2006,0.0,275,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1458,1459,20,9717,5,6,1950,1996,0.0,49,1029,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Define target vector
y = Xf["SalePrice"].ravel()
y[:5]

array([208500, 181500, 223500, 140000, 250000], dtype=int64)

In [12]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(dummies, y, random_state=78)


In [13]:
# Create the StandardScaler instance
scaler = StandardScaler()


In [14]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)


In [15]:
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [16]:
# Create the random forest classifier instance
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) #500 decision trees putting a vote
rf_model

RandomForestClassifier(n_estimators=500, random_state=78)

In [17]:
rf_model = rf_model.fit(X_train_scaled, y_train)

In [18]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [19]:
# Get the feature importance array
importances = rf_model.feature_importances_

In [20]:
# List the top 10 most important features
sorted(zip(rf_model.feature_importances_, X.columns[1:31]), reverse=True)

[(0.025706363360004723, 'MSSubClass'),
 (0.024825549011448865, 'OverallQual'),
 (0.02437659956591732, 'Foundation'),
 (0.024210708923580435, 'Condition2'),
 (0.02376271606316851, 'LotFrontage'),
 (0.023164891970868382, 'Condition1'),
 (0.0230456082757239, 'Neighborhood'),
 (0.020744002930578688, 'ExterQual'),
 (0.02022417635358984, 'LotShape'),
 (0.020011099896513982, 'LandContour'),
 (0.019103574035868726, 'LotConfig'),
 (0.017083688488116033, 'BsmtCond'),
 (0.01584390352205448, 'BsmtQual'),
 (0.015106243096804224, 'Exterior2nd'),
 (0.014203415593575826, 'Utilities'),
 (0.014095449794237401, 'BldgType'),
 (0.01265088365141831, 'LotArea'),
 (0.01018537779363353, 'RoofMatl'),
 (0.010012557293500706, 'MSZoning'),
 (0.009897688913151799, 'Street'),
 (0.008853728614390181, 'MasVnrArea'),
 (0.007464253352539215, 'OverallCond'),
 (0.006607740379641935, 'ExterCond'),
 (0.00629223939529302, 'BsmtExposure'),
 (0.006069366794839063, 'RoofStyle'),
 (0.0055053285308174745, 'YearRemodAdd'),
 (0.005

In [21]:
trained_data_sorted = trained_data.sort_values(by="MSSubClass", ascending=False)

home_sale_plot = trained_data_sorted.hvplot.bar(
    x="MSSubClass",
    y="SalePrice",
    title="Home Sale Price Based on Building Type"
)
home_sale_plot


:Bars   [MSSubClass]   (SalePrice)

In [22]:
trained_data_sorted = trained_data.sort_values(by="OverallQual", ascending=False)

home_sale_plot = trained_data_sorted.hvplot.bar(
    x="OverallQual",
    y="SalePrice",
    title="Home Sale Price Based on Overall Quality"
)
home_sale_plot


:Bars   [OverallQual]   (SalePrice)

In [23]:
trained_data_sorted = trained_data.sort_values(by="Foundation", ascending=False)

home_sale_plot = trained_data_sorted.hvplot.bar(
    x="Foundation",
    y="SalePrice",
    title="Home Sale Price Based on Foundation"
)
home_sale_plot


:Bars   [Foundation]   (SalePrice)

In [24]:
trained_data_sorted = trained_data.sort_values(by="Condition2", ascending=False)

home_sale_plot = trained_data_sorted.hvplot.bar(
    x="Condition2",
    y="SalePrice",
    title="Home Sale Price Based on Proximity to various conditions"
)
home_sale_plot


:Bars   [Condition2]   (SalePrice)

In [45]:
trained_data_cleaned = trained_data.dropna(subset = ['LotFrontage', 'SalePrice'])

In [46]:
trained_data_cleaned.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [47]:
#trained_data_sorted = trained_data.sort_values(by="LotFrontage", ascending=False)

home_sale_plot = trained_data_cleaned.hvplot.scatter(
    x="LotFrontage",
    y="SalePrice",
    title="Home Sale Price Based on Lot"
)
home_sale_plot


:Scatter   [LotFrontage]   (SalePrice)

In [48]:
# Reformat data of the independent variable X as a single-column array
X = trained_data_cleaned["LotFrontage"].values.reshape(-1, 1)

# Display sample data
X[:5]

array([[65.],
       [80.],
       [68.],
       [60.],
       [84.]])

In [49]:
X.shape

(1201, 1)

In [54]:
y = trained_data_cleaned["SalePrice"]

In [55]:
model.fit(X,y)

LinearRegression()

In [42]:
print(f"Model's slope: {model.coef_}")
print(f"Model's y-intercept: {model.intercept_}")
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

In [56]:
predicted_y_values = model.predict(X)

In [58]:
pricespredicted = trained_data_cleaned.copy()
pricespredicted["SalePrice_predicted"] = predicted_y_values
pricespredicted.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,SalePrice_predicted
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2008,WD,Normal,208500,174670.052202
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2007,WD,Normal,181500,192790.285440
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2008,WD,Normal,223500,178294.098849
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000,168629.974456
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,12,2008,WD,Normal,250000,197622.347636


In [62]:
best_fit_line = pricespredicted.hvplot.line(
    x="LotFrontage",
    y="SalePrice_predicted",
    color="red"
)
best_fit_line

:Curve   [LotFrontage]   (SalePrice_predicted)

In [64]:
home_sale_plot * best_fit_line

:Overlay
   .Scatter.I :Scatter   [LotFrontage]   (SalePrice)
   .Curve.I   :Curve   [LotFrontage]   (SalePrice_predicted)

In [26]:
trained_data_sorted = trained_data.sort_values(by="Condition1", ascending=False)

home_sale_plot = trained_data_sorted.hvplot.bar(
    x="Condition1",
    y="SalePrice",
    title="Home Sale Price Based on proxmity to main road or railroad"
)
home_sale_plot

#condition 1 and 2 are together basically

:Bars   [Condition1]   (SalePrice)

In [27]:
trained_data_sorted = trained_data.sort_values(by="Neighborhood", ascending=False)

home_sale_plot = trained_data_sorted.hvplot.bar(
    x="Neighborhood",
    y="SalePrice",
    title="Home Sale Price Based on Neighborhood",
)
home_sale_plot.opts(xrotation=45)

home_sale_plot



:Bars   [Neighborhood]   (SalePrice)

In [28]:
trained_data_sorted = trained_data.sort_values(by="ExterQual", ascending=False)

home_sale_plot = trained_data_sorted.hvplot.bar(
    x="ExterQual",
    y="SalePrice",
    title="Home Sale Price Based on Exterior Quality",
)

home_sale_plot


:Bars   [ExterQual]   (SalePrice)

In [29]:
trained_data_sorted = trained_data.sort_values(by="LotShape", ascending=False)

home_sale_plot = trained_data_sorted.hvplot.bar(
    x="LotShape",
    y="SalePrice",
    title="Home Sale Price Based on Shape of Property",
)

home_sale_plot


:Bars   [LotShape]   (SalePrice)

In [30]:
trained_data_sorted = trained_data.sort_values(by="LandContour", ascending=False)

home_sale_plot = trained_data_sorted.hvplot.bar(
    x="LandContour",
    y="SalePrice",
    title="Home Sale Price Based on Flatness of property",
)

home_sale_plot


:Bars   [LandContour]   (SalePrice)

In [31]:
trained_data_sorted = trained_data.sort_values(by="Heating", ascending=False)

home_sale_plot = trained_data_sorted.hvplot.bar(
    x="Heating",
    y="SalePrice",
    title="Home Sale Price Based on Heating type",
)

home_sale_plot


:Bars   [Heating]   (SalePrice)